In [1]:
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
import sonia
from sonnia.sonnia import SoNNia
from sonia.evaluate_model import EvaluateModel
from sonia.plotting import Plotter
from sonia.sequence_generation import SequenceGeneration
from sonnia.processing import Processing
from scipy import stats
import matplotlib.pyplot as plt
import olga.load_model as olga_load_model
import olga.generation_probability as pgen
import os
import olga.sequence_generation as seq_gen
import tensorflow.keras.backend as K
import tensorflow as tf
from sonnia.utils import sample_olga

2021-12-09 19:37:52.530655: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:
2021-12-09 19:37:52.530687: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### TCR Sharing

In [2]:
fInput = 'tcell_adaptive_cdr3aa_total_counts.tar.gz'
f1 = pd.read_csv(fInput, delimiter='\t')
counts = np.array(f1['counts'])

In [3]:
hist,_ = np.histogram(counts, bins = np.arange(1,1416))

In [5]:
fN = pd.read_csv('inferred_n.txt', delimiter='\n')
N1 = list(fN['Inferred_N'])

In [7]:
# Coefficients with q_healthy = 0.472

f2 = pd.read_csv('generated_ppost_tcr.txt', delimiter='\t')
f2 = f2.dropna(subset=['Ppost'])
f2 = f2[f2.Ppost > 1e-50]
aa_hist, bins_aa = np.histogram(-np.log(f2.Ppost), bins=(1000), density=True)

E_pp = bins_aa[:-1]
delta=E_pp[2]-E_pp[1]
P_pp = aa_hist
exp_pp = np.exp(E_pp)
g_pp = np.multiply(P_pp, exp_pp)

q=0.472
f=0.236

k=0
integrand1=[]
i=0
for i in range(320):
    d=np.poly1d([0,1])
    for k in range(len(N1)):
        a=b=0
        a = 1-np.exp(-np.exp(-E_pp[i])*N1[k]/(f*q))
        b = np.exp(-np.exp(-E_pp[i])*N1[k]/(f*q))
        c = np.poly1d([a,b])
        d=np.convolve(d,c)
        coeff1=d
    integrand1.append(g_pp[i]*f*q*delta*coeff1)
    
k=0
integrand2=[]
j=0
for j in range(320,380):
    m=np.poly1d([0,1])
    for k in range(len(N1)):
        g=h=0
        g = N1[k]*np.exp(-E_pp[j]) - ((N1[k]**2)*np.exp(-2*E_pp[j]))/(2*f*q)
        h = 1-(N1[k]*np.exp(-E_pp[j]) + ((N1[k]**2)*np.exp(-2*E_pp[j]))/(2*f*q))
        l = np.poly1d([g,h])
        m=np.convolve(m,l)
        coeff2=m
    integrand2.append(g_pp[j]*delta*coeff2)
    
coefficients = 0
coefficients = sum(integrand1)
coefficients2 = 0
coefficients2 = sum(integrand2)

u = list(np.flip(coefficients)[1:1390])
v = list(np.flip(coefficients2)[1390:])


In [7]:
Mpost = np.array(u+v)
snumber = np.array(range(1,1415))

Mpost_error = []
for element in Mpost:
    Mpost_error.append(np.sqrt(element))

In [8]:
# Coefficients with q_covid19 = 0.452

q=0.452
f=0.236

k=0
integrand1=[]
i=0
for i in range(320):
    d=np.poly1d([0,1])
    for k in range(len(N1)):
        a=b=0
        a = 1-np.exp(-np.exp(-E_pp[i])*N1[k]/(f*q))
        b = np.exp(-np.exp(-E_pp[i])*N1[k]/(f*q))
        c = np.poly1d([a,b])
        d=np.convolve(d,c)
        coeff1=d
    integrand1.append(g_pp[i]*f*q*delta*coeff1)
    
k=0
integrand2=[]
j=0
for j in range(320,380):
    m=np.poly1d([0,1])
    for k in range(len(N1)):
        g=h=0
        g = N1[k]*np.exp(-E_pp[j]) - ((N1[k]**2)*np.exp(-2*E_pp[j]))/(2*f*q)
        h = 1-(N1[k]*np.exp(-E_pp[j]) + ((N1[k]**2)*np.exp(-2*E_pp[j]))/(2*f*q))
        l = np.poly1d([g,h])
        m=np.convolve(m,l)
        coeff2=m
    integrand2.append(g_pp[j]*delta*coeff2)
    
coefficients = 0
coefficients = sum(integrand1)
coefficients2 = 0
coefficients2 = sum(integrand2)

u = list(np.flip(coefficients)[1:1390])
v = list(np.flip(coefficients2)[1390:])

In [9]:
Mpost_sel = np.array(u+v)
snumber = np.array(range(1,1415))

Mpost_sel_error = []
for element in Mpost_sel:
    Mpost_sel_error.append(np.sqrt(element))

In [14]:
snumber = []
for i in range(1,1415):
    snumber.append(i)

In [16]:
df_print = pd.DataFrame(list(zip(snumber, hist, Mpost_sel, Mpost_sel_error)),
               columns =['Sharing Number','Data', 'DoS+q=0.452','DoS+q=0.452 - Error'])

### Significance Plot

In [2]:
fInput = '/home/ruiz/Scripts/Paper_figures/Figure5/Figure 5B/cdr3aa_vj_files_adaptive.tar.gz'
data = pd.read_csv(fInput,delimiter='\t',compression='gzip')

In [3]:
df2 = data.loc[((data['p']==0.0)|(data['p']<1e-140))&(np.log10(data['ppost_f']/0.452)/np.log10(data['pdata'])>1.1), :]

In [11]:
ppost = data['ppost_f']/0.452
pdata = data['pdata']
# Only for drawing
pdata2 = data.loc[(data['Posterior_prob']==0.0)&(np.log10(data['ppost_f'])/np.log10(data['pdata'])>1.25), ['pdata']].values
ppost2 = data.loc[(data['Posterior_prob']==0.0)&(np.log10(data['ppost_f'])/np.log10(data['pdata'])>1.25), ['ppost_f']].values/0.452

In [12]:
x1 = np.log10(pdata)
y1 = np.log10(ppost)
x2 = np.log10(pdata2)
y2 = np.log10(ppost2)

In [13]:
del(data)

### Model comparison

In [4]:
main_folder='/home/ruiz/bin/olga3/olga/default_models/human_T_beta/'

In [5]:
# Adaptive Sample 1 

model_folder1 = '/home/ruiz/Selection/Tcells_Analysis/soNNia_model_Covid19/1/'
qm_adap1 = SoNNia(load_dir=model_folder1, custom_pgen_model=main_folder,chain_type='human_T_beta',min_energy_clip=-7,l2_reg=1e-4)

initial_adap1=np.array([s[0][0] for s in qm_adap1.features])
l_length_adap1=len(np.arange(len(initial_adap1))[initial_adap1=='l'])
a_length_adap1=len(np.arange(len(initial_adap1))[initial_adap1=='a'])
vj_length_adap1=len(np.arange(len(initial_adap1))[initial_adap1=='v'])+len(np.arange(len(initial_adap1))[initial_adap1=='j'])
vj_features_adap1=np.array(qm_adap1.features[-vj_length_adap1:])
v_genes_adap1=[]
j_genes_adap1=[]
for feat in vj_features_adap1:
    if (feat[0][0]=='v')==True:
        v_genes_adap1.append(feat[0])
    if (feat[0][0]=='j')==True:
        j_genes_adap1.append(feat[0])
vj_data_marginals_adap1=np.array(qm_adap1.data_marginals[-vj_length_adap1:])

v_genes_adap1 = [x.replace('v', 'IGHV') for x in v_genes_adap1]
j_genes_adap1 = [x.replace('j', 'IGHJ') for x in j_genes_adap1]
orderv_adap1=np.argsort(vj_data_marginals_adap1[:len(v_genes_adap1)])
orderj_adap1=np.argsort(vj_data_marginals_adap1[len(v_genes_adap1):])
df_PV_adap1 = pd.Series(dict(zip(np.array(v_genes_adap1)[orderv_adap1], vj_data_marginals_adap1[:len(v_genes_adap1)][orderv_adap1]))).to_frame("pV")
df_PJ_adap1 = pd.Series(dict(zip(np.array(j_genes_adap1)[orderj_adap1], vj_data_marginals_adap1[len(v_genes_adap1):][orderj_adap1]))).to_frame("pJ")

# Adaptive Sample 2 

model_folder2 = '/home/ruiz/Selection/Tcells_Analysis/soNNia_model_Covid19/2/'
qm_adap2 = SoNNia(load_dir=model_folder2, custom_pgen_model=main_folder,chain_type='human_T_beta',min_energy_clip=-7,l2_reg=1e-4)

initial_adap2=np.array([s[0][0] for s in qm_adap2.features])
l_length_adap2=len(np.arange(len(initial_adap2))[initial_adap2=='l'])
a_length_adap2=len(np.arange(len(initial_adap2))[initial_adap2=='a'])
vj_length_adap2=len(np.arange(len(initial_adap2))[initial_adap2=='v'])+len(np.arange(len(initial_adap2))[initial_adap2=='j'])
vj_features_adap2=np.array(qm_adap2.features[-vj_length_adap2:])
v_genes_adap2=[]
j_genes_adap2=[]
for feat in vj_features_adap2:
    if (feat[0][0]=='v')==True:
        v_genes_adap2.append(feat[0])
    if (feat[0][0]=='j')==True:
        j_genes_adap2.append(feat[0])
vj_data_marginals_adap2=np.array(qm_adap2.data_marginals[-vj_length_adap2:])

v_genes_adap2 = [x.replace('v', 'IGHV') for x in v_genes_adap2]
j_genes_adap2 = [x.replace('j', 'IGHJ') for x in j_genes_adap2]
orderv_adap2=np.argsort(vj_data_marginals_adap2[:len(v_genes_adap2)])
orderj_adap2=np.argsort(vj_data_marginals_adap2[len(v_genes_adap2):])
df_PV_adap2 = pd.Series(dict(zip(np.array(v_genes_adap2)[orderv_adap2], vj_data_marginals_adap2[:len(v_genes_adap2)][orderv_adap2]))).to_frame("pV")
df_PJ_adap2 = pd.Series(dict(zip(np.array(j_genes_adap2)[orderj_adap2], vj_data_marginals_adap2[len(v_genes_adap2):][orderj_adap2]))).to_frame("pJ")

# Adaptive Sample 3

model_folder3 = '/home/ruiz/Selection/Tcells_Analysis/soNNia_model_Covid19/3/'
qm_adap3 = SoNNia(load_dir=model_folder3, custom_pgen_model=main_folder,chain_type='human_T_beta',min_energy_clip=-7,l2_reg=1e-4)

initial_adap3=np.array([s[0][0] for s in qm_adap3.features])
l_length_adap3=len(np.arange(len(initial_adap3))[initial_adap3=='l'])
a_length_adap3=len(np.arange(len(initial_adap3))[initial_adap3=='a'])
vj_length_adap3=len(np.arange(len(initial_adap3))[initial_adap3=='v'])+len(np.arange(len(initial_adap3))[initial_adap3=='j'])
vj_features_adap3=np.array(qm_adap3.features[-vj_length_adap3:])
v_genes_adap3=[]
j_genes_adap3=[]
for feat in vj_features_adap3:
    if (feat[0][0]=='v')==True:
        v_genes_adap3.append(feat[0])
    if (feat[0][0]=='j')==True:
        j_genes_adap3.append(feat[0])
vj_data_marginals_adap3=np.array(qm_adap3.data_marginals[-vj_length_adap3:])

v_genes_adap3 = [x.replace('v', 'IGHV') for x in v_genes_adap3]
j_genes_adap3 = [x.replace('j', 'IGHJ') for x in j_genes_adap3]
orderv_adap3=np.argsort(vj_data_marginals_adap3[:len(v_genes_adap3)])
orderj_adap3=np.argsort(vj_data_marginals_adap3[len(v_genes_adap3):])
df_PV_adap3 = pd.Series(dict(zip(np.array(v_genes_adap3)[orderv_adap3], vj_data_marginals_adap3[:len(v_genes_adap3)][orderv_adap3]))).to_frame("pV")
df_PJ_adap3 = pd.Series(dict(zip(np.array(j_genes_adap3)[orderj_adap3], vj_data_marginals_adap3[len(v_genes_adap3):][orderj_adap3]))).to_frame("pJ")

2021-12-09 19:39:40.711563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:
2021-12-09 19:39:40.711592: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-09 19:39:40.711616: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (zuzia): /proc/driver/nvidia/version does not exist
2021-12-09 19:39:40.711854: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Cannot find data_seqs.tsv  --  no data seqs loaded.
Cannot find gen_seqs.tsv  --  no generated seqs loaded.
Cannot find data_seqs.tsv  --  no data seqs loaded.
Cannot find gen_seqs.tsv  --  no generated seqs loaded.
Cannot find data_seqs.tsv  --  no data seqs loaded.
Cannot find gen_seqs.tsv  --  no generated seqs loaded.


In [6]:
# Emerson Sample 1

model_folder4 = '/home/ruiz/Selection/Tcells_Analysis/soNNia_model_healthy/1/'
qm_emer1 = SoNNia(load_dir=model_folder4, custom_pgen_model=main_folder,chain_type='human_T_beta',min_energy_clip=-7,l2_reg=1e-4)

initial_emer1=np.array([s[0][0] for s in qm_emer1.features])
l_length_emer1=len(np.arange(len(initial_emer1))[initial_emer1=='l'])
a_length_emer1=len(np.arange(len(initial_emer1))[initial_emer1=='a'])
vj_length_emer1=len(np.arange(len(initial_emer1))[initial_emer1=='v'])+len(np.arange(len(initial_emer1))[initial_emer1=='j'])
vj_features_emer1=np.array(qm_emer1.features[-vj_length_emer1:])
v_genes_emer1=[]
j_genes_emer1=[]
for feat in vj_features_emer1:
    if (feat[0][0]=='v')==True:
        v_genes_emer1.append(feat[0])
    if (feat[0][0]=='j')==True:
        j_genes_emer1.append(feat[0])
vj_data_marginals_emer1=np.array(qm_emer1.data_marginals[-vj_length_emer1:])

v_genes_emer1 = [x.replace('v', 'IGHV') for x in v_genes_emer1]
j_genes_emer1 = [x.replace('j', 'IGHJ') for x in j_genes_emer1]
orderv_emer1=np.argsort(vj_data_marginals_emer1[:len(v_genes_emer1)])
orderj_emer1=np.argsort(vj_data_marginals_emer1[len(v_genes_emer1):])
df_PV_emer1 = pd.Series(dict(zip(np.array(v_genes_emer1)[orderv_emer1], vj_data_marginals_emer1[:len(v_genes_emer1)][orderv_emer1]))).to_frame("pV")
df_PJ_emer1 = pd.Series(dict(zip(np.array(j_genes_emer1)[orderj_emer1], vj_data_marginals_emer1[len(v_genes_emer1):][orderj_emer1]))).to_frame("pJ")

# Emerson Sample 2

model_folder5 = '/home/ruiz/Selection/Tcells_Analysis/soNNia_model_healthy/2/'
qm_emer2 = SoNNia(load_dir=model_folder5, custom_pgen_model=main_folder,chain_type='human_T_beta',min_energy_clip=-7,l2_reg=1e-4)

initial_emer2=np.array([s[0][0] for s in qm_emer2.features])
l_length_emer2=len(np.arange(len(initial_emer2))[initial_emer2=='l'])
a_length_emer2=len(np.arange(len(initial_emer2))[initial_emer2=='a'])
vj_length_emer2=len(np.arange(len(initial_emer2))[initial_emer2=='v'])+len(np.arange(len(initial_emer2))[initial_emer2=='j'])
vj_features_emer2=np.array(qm_emer2.features[-vj_length_emer2:])
v_genes_emer2=[]
j_genes_emer2=[]
for feat in vj_features_emer2:
    if (feat[0][0]=='v')==True:
        v_genes_emer2.append(feat[0])
    if (feat[0][0]=='j')==True:
        j_genes_emer2.append(feat[0])
vj_data_marginals_emer2=np.array(qm_emer2.data_marginals[-vj_length_emer2:])

v_genes_emer2 = [x.replace('v', 'IGHV') for x in v_genes_emer2]
j_genes_emer2 = [x.replace('j', 'IGHJ') for x in j_genes_emer2]
orderv_emer2=np.argsort(vj_data_marginals_emer2[:len(v_genes_emer2)])
orderj_emer2=np.argsort(vj_data_marginals_emer2[len(v_genes_emer2):])
df_PV_emer2 = pd.Series(dict(zip(np.array(v_genes_emer2)[orderv_emer2], vj_data_marginals_emer2[:len(v_genes_emer2)][orderv_emer2]))).to_frame("pV")
df_PJ_emer2 = pd.Series(dict(zip(np.array(j_genes_emer2)[orderj_emer2], vj_data_marginals_emer2[len(v_genes_emer2):][orderj_emer2]))).to_frame("pJ")

# Emerson Sample 3

model_folder6 = '/home/ruiz/Selection/Tcells_Analysis/soNNia_model_healthy/3/'
qm_emer3 = SoNNia(load_dir=model_folder6, custom_pgen_model=main_folder,chain_type='human_T_beta',min_energy_clip=-7,l2_reg=1e-4)

initial_emer3=np.array([s[0][0] for s in qm_emer3.features])
l_length_emer3=len(np.arange(len(initial_emer3))[initial_emer3=='l'])
a_length_emer3=len(np.arange(len(initial_emer3))[initial_emer3=='a'])
vj_length_emer3=len(np.arange(len(initial_emer3))[initial_emer3=='v'])+len(np.arange(len(initial_emer3))[initial_emer3=='j'])
vj_features_emer3=np.array(qm_emer3.features[-vj_length_emer3:])
v_genes_emer3=[]
j_genes_emer3=[]
for feat in vj_features_emer3:
    if (feat[0][0]=='v')==True:
        v_genes_emer3.append(feat[0])
    if (feat[0][0]=='j')==True:
        j_genes_emer3.append(feat[0])
vj_data_marginals_emer3=np.array(qm_emer3.data_marginals[-vj_length_emer3:])

v_genes_emer3 = [x.replace('v', 'IGHV') for x in v_genes_emer3]
j_genes_emer3 = [x.replace('j', 'IGHJ') for x in j_genes_emer3]
orderv_emer3=np.argsort(vj_data_marginals_emer3[:len(v_genes_emer3)])
orderj_emer3=np.argsort(vj_data_marginals_emer3[len(v_genes_emer3):])
df_PV_emer3 = pd.Series(dict(zip(np.array(v_genes_emer3)[orderv_emer3], vj_data_marginals_emer3[:len(v_genes_emer3)][orderv_emer3]))).to_frame("pV")
df_PJ_emer3 = pd.Series(dict(zip(np.array(j_genes_emer3)[orderj_emer3], vj_data_marginals_emer3[len(v_genes_emer3):][orderj_emer3]))).to_frame("pJ")


Cannot find data_seqs.tsv  --  no data seqs loaded.
Cannot find gen_seqs.tsv  --  no generated seqs loaded.
Cannot find data_seqs.tsv  --  no data seqs loaded.
Cannot find gen_seqs.tsv  --  no generated seqs loaded.
Cannot find data_seqs.tsv  --  no data seqs loaded.
Cannot find gen_seqs.tsv  --  no generated seqs loaded.


In [7]:
# Covid19 significant

model_folder7 = '/home/ruiz/Selection/Tcells_Analysis/soNNia_model_Covid19/Significant/'
qm_adap_s = SoNNia(load_dir=model_folder7, custom_pgen_model=main_folder,chain_type='human_T_beta',min_energy_clip=-7,l2_reg=1e-4)

initial_adap_s=np.array([s[0][0] for s in qm_adap_s.features])
l_length_adap_s=len(np.arange(len(initial_adap_s))[initial_adap_s=='l'])
a_length_adap_s=len(np.arange(len(initial_adap_s))[initial_adap_s=='a'])
vj_length_adap_s=len(np.arange(len(initial_adap_s))[initial_adap_s=='v'])+len(np.arange(len(initial_adap_s))[initial_adap_s=='j'])
vj_features_adap_s=np.array(qm_adap_s.features[-vj_length_adap_s:])
v_genes_adap_s=[]
j_genes_adap_s=[]
for feat in vj_features_adap_s:
    if (feat[0][0]=='v')==True:
        v_genes_adap_s.append(feat[0])
    if (feat[0][0]=='j')==True:
        j_genes_adap_s.append(feat[0])
vj_data_marginals_adap_s=np.array(qm_adap_s.data_marginals[-vj_length_adap_s:])

v_genes_adap_s = [x.replace('v', 'IGHV') for x in v_genes_adap_s]
j_genes_adap_s = [x.replace('j', 'IGHJ') for x in j_genes_adap_s]
orderv_adap_s=np.argsort(vj_data_marginals_adap_s[:len(v_genes_adap_s)])
orderj_adap_s=np.argsort(vj_data_marginals_adap_s[len(v_genes_adap_s):])
df_PV_adap_s = pd.Series(dict(zip(np.array(v_genes_adap_s)[orderv_adap_s], vj_data_marginals_adap_s[:len(v_genes_adap_s)][orderv_adap_s]))).to_frame("pV")
df_PJ_adap_s = pd.Series(dict(zip(np.array(j_genes_adap_s)[orderj_adap_s], vj_data_marginals_adap_s[len(v_genes_adap_s):][orderj_adap_s]))).to_frame("pJ")

Cannot find data_seqs.tsv  --  no data seqs loaded.
Cannot find gen_seqs.tsv  --  no generated seqs loaded.


In [8]:
# Healthy significant

model_folder8 = '/home/ruiz/Selection/Tcells_Analysis/soNNia_model_healthy/Significant/'
qm_emer_s = SoNNia(load_dir=model_folder8, custom_pgen_model=main_folder,chain_type='human_T_beta',min_energy_clip=-7,l2_reg=1e-4)

initial_emer_s=np.array([s[0][0] for s in qm_emer_s.features])
l_length_emer_s=len(np.arange(len(initial_emer_s))[initial_emer_s=='l'])
a_length_emer_s=len(np.arange(len(initial_emer_s))[initial_emer_s=='a'])
vj_length_emer_s=len(np.arange(len(initial_emer_s))[initial_emer_s=='v'])+len(np.arange(len(initial_emer_s))[initial_emer_s=='j'])
vj_features_emer_s=np.array(qm_emer_s.features[-vj_length_emer_s:])
v_genes_emer_s=[]
j_genes_emer_s=[]
for feat in vj_features_emer_s:
    if (feat[0][0]=='v')==True:
        v_genes_emer_s.append(feat[0])
    if (feat[0][0]=='j')==True:
        j_genes_emer_s.append(feat[0])
vj_data_marginals_emer_s=np.array(qm_emer_s.data_marginals[-vj_length_emer_s:])

Cannot find data_seqs.tsv  --  no data seqs loaded.
Cannot find gen_seqs.tsv  --  no generated seqs loaded.


In [9]:
listOfStrings1 = ['Covid-19' for i in range(59)]
listOfStrings2 = ['Healthy' for i in range(59)]
listOfStrings3 = ['Covid-19 Significant' for i in range(59)]

df_PV_adap1['diagnosis'] = listOfStrings1
df_PV_adap2['diagnosis'] = listOfStrings1
df_PV_adap3['diagnosis'] = listOfStrings1
df_PV_emer1['diagnosis'] = listOfStrings2
df_PV_emer2['diagnosis'] = listOfStrings2
df_PV_emer3['diagnosis'] = listOfStrings2
df_PV_adap_s['diagnosis'] = listOfStrings3

df_PV_f1 = df_PV_emer1.append(df_PV_emer2)
df_PV_f2 = df_PV_f1.append(df_PV_emer3)
df_PV_f3 = df_PV_f2.append(df_PV_adap1)
df_PV_f4 = df_PV_f3.append(df_PV_adap2)
df_PV_f5 = df_PV_f4.append(df_PV_adap3)
df_PV_f = df_PV_f5.append(df_PV_adap_s)

In [10]:
listOfStrings4 = ['Covid-19 Significant' for i in range(13)]
listOfStrings5 = ['Covid-19' for i in range(13)]
listOfStrings6 = ['Healthy' for i in range(13)]

df_PJ_adap1['diagnosis'] = listOfStrings5
df_PJ_adap2['diagnosis'] = listOfStrings5
df_PJ_adap3['diagnosis'] = listOfStrings5
df_PJ_adap_s['diagnosis'] = listOfStrings4
df_PJ_emer1['diagnosis'] = listOfStrings6
df_PJ_emer2['diagnosis'] = listOfStrings6
df_PJ_emer3['diagnosis'] = listOfStrings6

df_PJ_f1 = df_PJ_emer1.append(df_PJ_emer2)
df_PJ_f2 = df_PJ_f1.append(df_PJ_emer3)
df_PJ_f3 = df_PJ_f2.append(df_PJ_adap1)
df_PJ_f4 = df_PJ_f3.append(df_PJ_adap2)
df_PJ_f5 = df_PJ_f4.append(df_PJ_adap3)
df_PJ_f = df_PJ_f5.append(df_PJ_adap_s)

In [11]:
# CDR3 Length

data_c = np.array([qm_adap1.data_marginals[:l_length_adap1],qm_adap2.data_marginals[:l_length_adap2],qm_adap3.data_marginals[:l_length_adap3]])
mean_data_c = np.mean(data_c, axis=0)
std_data_c = np.std(data_c, axis=0)

data_h= np.array([qm_emer1.data_marginals[:l_length_emer1],qm_emer2.data_marginals[:l_length_emer2],qm_emer3.data_marginals[:l_length_emer3]])
mean_data_h = np.mean(data_h, axis=0)
std_data_h = np.std(data_h, axis=0)

data_s = np.array(qm_adap_s.data_marginals[:l_length_adap_s])

data_emer_s = np.array(qm_emer_s.data_marginals[:l_length_emer_s])

### All plots

In [22]:
font={'size': 10, 'weight': 'normal', 'family': 'DejaVu Sans'}
plt.rc('text', usetex=True)
plt.rc('font', **font)

In [ ]:
fig =  plt.figure(figsize=(9,12))
plt.rc('text', usetex=True)

#Plot 1

kwargs = {'xscale':'log', 'yscale':'log', 'xlabel':'Sharing number', 'ylabel':'Number of unique CDR3s'}

ax1 = plt.subplot(4,4,(1,2), **kwargs)

ax1.plot(np.arange(1, 1415), hist,'+',color=sns.color_palette("Paired")[1], markersize=8, mew=2, label='COVID-19')

ax1.plot(snumber, Mpost, color=sns.color_palette("Paired")[5],label='$P_{\mathrm{post}}$ \n + healthy selection')
ax1.fill_between(snumber,Mpost-Mpost_error,Mpost+Mpost_error,alpha=0.3,color='lightcoral')

ax1.plot(snumber, Mpost_sel, color=sns.color_palette("Paired")[7],label='$P_{\mathrm{post}}$ \n + COVID-19 selection')
ax1.fill_between(snumber,Mpost_sel-Mpost_sel_error,Mpost_sel+Mpost_sel_error,alpha=0.7,color=sns.color_palette("Paired")[6])
 
ax1.set_xlim([0.5,1500])
ax1.set_ylim([0.5,5e8])
ax1.set_title(r'Human TCR$\beta$')
ax1.legend(frameon=False)

#Plot 2

kwargs = {'xlabel':'$\log_{10} P_{\mathrm{data}}$', 'ylabel':'$\log_{10} P^*_{\mathrm{post}}$'}

ax2 = plt.subplot(4,4,(3,4), **kwargs)

x = [x1[:-1], x2]
y = [y1[:-1], y2]

data, x_e, y_e = np.histogram2d( x[0], y[0], bins = [50,50], density = True )
z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x[0],y[0]]).T , method = "splinef2d", bounds_error = False)

z[np.where(np.isnan(z))] = 0.0

idx = z.argsort()
x[0], y[0], z = x[0][idx], y[0][idx], z[idx]

ax2.scatter( x[0], y[0], c=z, s=2, rasterized=True)
ax2.scatter(x[1],y[1], color='red',s=2, rasterized=True)

# Plot 3


ax3 = plt.subplot(4,4,(7,8))

ax3.plot(np.arange(l_length_emer1),mean_data_h,color = sns.color_palette("Set2")[0],linewidth=.6, label='Healthy',alpha=0.7)
ax3.fill_between(np.arange(l_length_emer1),mean_data_h-2*std_data_h, mean_data_h+2*std_data_h,color = sns.color_palette("Set2")[0],alpha=0.5)
ax3.plot(np.arange(l_length_emer_s),data_emer_s,color = sns.color_palette("tab10")[0],linewidth=.6, label='Healthy \n sign.',alpha=1)
ax3.plot(np.arange(l_length_adap1),mean_data_c,color = sns.color_palette("Set2")[1],linewidth=.6, label='Covid-19',alpha=0.9)
ax3.fill_between(np.arange(l_length_adap1),mean_data_c-std_data_c, mean_data_c+std_data_c,color = sns.color_palette("Set2")[1],alpha=0.2)
ax3.plot(np.arange(l_length_adap_s),data_s,color = sns.color_palette("Set2")[2],linewidth=.6, label='Covid-19 \n sign.',alpha=0.9)

ax3.legend(loc='upper left', title=None,frameon=False)
ax3.set_xlabel('CDRH3 length [aa]')
ax3.set_ylabel('Relative counts')

ax3.set_xlim(8,20)

plt.setp(ax3.spines.values(), linewidth=.6)

ax3.xaxis.set_tick_params(width=.6)
ax3.yaxis.set_tick_params(width=.6)


# Plot 5

ax5 = plt.subplot(4,4,(11,12))

df_sel_J = df_PJ_f.loc[df_PJ_f['pJ']>0.001,:]

g = sns.barplot(x=df_sel_J.index, y=df_sel_J.pJ, hue=df_sel_J.diagnosis, palette='Set2', ax=ax5,
            errwidth=0.6,ci = "sd", capsize = 0.2)

# statistical annotation
x15, x16 = 9.5,10.5
y7,h,col = 0.15, 0.003, 'k'
x17, x18 = 10.5,11.5
x19, x20 = 11.5,12.5
y8 = 0.225

plt.plot([x15, x15, x16, x16], [y7, y7+h, y7+h, y7], lw=.6, c=col)
plt.text((x15+x16)*.5, y7+h, '**', ha='center', va='bottom', color=col)
plt.plot([x17, x17, x18, x18], [y8, y8+h, y8+h, y8], lw=.6, c=col)
plt.text((x17+x18)*.5, y8+h, '**', ha='center', va='bottom', color=col)
plt.plot([x19, x19, x20, x20], [y8, y8+h, y8+h, y8], lw=.6, c=col)
plt.text((x19+x20)*.5, y8+h, '**', ha='center', va='bottom', color=col)

ax5.set_ylabel('Relative counts')
plt.xticks(rotation='vertical')
ax5.legend(loc='upper right',frameon=False)
ax5.set_ylim(0,0.7)

plt.setp(ax5.spines.values(), linewidth=.6)

ax5.xaxis.set_tick_params(width=.6)
ax5.yaxis.set_tick_params(width=.6)

# Plot 6


ax6 = plt.subplot(4,4,(13,16))

df_sel_V = df_PV_f.loc[df_PV_f['pV']>0.001,:]
g = sns.barplot(x=df_sel_V.index, y=df_sel_V.pV, hue=df_sel_V.diagnosis, palette='Set2', ax=ax6,
            errwidth=0.6,ci = "sd", capsize = 0.2)

# statistical annotation
x3, x4 = 30.5,31.5
x5, x6 = 27.5,28.5
y3,h,col = 0.127, 0.001, 'k'
y4 = 0.04
x7, x8 = 40.5, 41.5
y5 = 0.09
x9, x10 = 39.5, 40.5
x11, x12 = 32.5, 33.5
y6 = 0.06
x13, x14 = 37.5, 38.5

plt.plot([x3, x3, x4, x4], [y3, y3+h, y3+h, y3], lw=.6, c=col)
plt.text((x3+x4)*.5, y3+h, '**', ha='center', va='bottom', color=col)
plt.plot([x5, x5, x6, x6], [y4, y4+h, y4+h, y4], lw=.6, c=col)
plt.text((x5+x6)*.5, y4+h, '**', ha='center', va='bottom', color=col)
plt.plot([x7, x7, x8, x8], [y5, y5+h, y5+h, y5], lw=.6, c=col)
plt.text((x7+x8)*.5, y5+h, '**', ha='center', va='bottom', color=col)
plt.plot([x9, x9, x10, x10], [y5, y5+h, y5+h, y5], lw=.6, c=col)
plt.text((x9+x10)*.5, y5+h, '**', ha='center', va='bottom', color=col)
plt.plot([x11, x11, x12, x12], [y6, y6+h, y6+h, y6], lw=.6, c=col)
plt.text((x11+x12)*.5, y6+h, '**', ha='center', va='bottom', color=col)
plt.plot([x13, x13, x14, x14], [y6, y6+h, y6+h, y6], lw=.6, c=col)
plt.text((x13+x14)*.5, y6+h, '**', ha='center', va='bottom', color=col)

plt.xticks(rotation='vertical')
ax6.set_ylabel("Relative counts")
ax6.set_ylim(0,0.14)
g.legend_.remove()

plt.tight_layout()

fig.savefig('/home/ruiz/Figure4a.pdf',bbox_inches='tight')

plt.show()